# Libraries

In [33]:
import pandas as pd
from getpass import getuser
from collections import defaultdict
from datetime import datetime, timedelta
import re

# Load and inspect dataset

In [34]:
# Get the current user's name
user = getuser()

In [35]:

# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\false_start\data\in\kate\2001_zurich.xlsx'

# Read the dataset with a different encoding
df = pd.read_excel(data_path)

display(df.head())

,2001 Weltklasse Zurich,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Result List 400m Hurdles Men B-Series 17.08...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,World Record,NaN,NaN,NaN,NaN,NaN,46.78,NaN,Kevin Young,NaN,...,NaN,Barcelone,NaN,NaN,NaN,NaN,NaN,1992-08-06,NaN,NaN
2,Meeting Record,NaN,NaN,NaN,NaN,NaN,47.10,NaN,Samuel Matete,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1991-08-07,NaN,NaN
3,rank,BIB,athlete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,finish time,NaN,NaN,NaN,NaN,NaT,NaN,reaction time
4,1,213,"GORBAN, Boris",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,48.58,NaN,NaN,NaN,NaN,NaT,NaN,0.262


In [36]:

# def extract_table_info(df):
#     columns_of_interest = ['rank', 'BIB', 'athlete', 'nat', 'finish time', 'reaction time']
#     tables = []
#     metadata_list = []

#     # Extract judge's name once from the entire dataset
#     judge_info_row = df[df.iloc[:, 0].astype(str).str.contains('judge:', na=False)]
#     judge = None
#     if not judge_info_row.empty:
#         judge_wind_info = str(judge_info_row.iloc[0, 0])  # First occurrence
#         if "judge:" in judge_wind_info:
#             judge = judge_wind_info.split("judge:")[1].split("wind:")[0].strip().split("(")[0].strip()
#     print(f"Extracted Judge: {judge}")  # Debugging Output

#     start_indices = df[df.iloc[:, 0].astype(str).str.contains('rank', na=False)].index
#     end_indices = df[df.iloc[:, 0].astype(str).str.contains('judge', na=False)].index

#     for start, end in zip(start_indices, end_indices):
#         print(f"\nProcessing table from row {start} to {end}")

#         # Extract metadata
#         metadata_row = df.iloc[start-3:start].reset_index(drop=True)
#         first_row = metadata_row.iloc[0, 0].replace("Result List", "").strip()
#         print(f"First Row Extracted: {first_row}")

#         # Extract date from last two parts of the string
#         date_match = re.search(r"\d{2}\.\d{2}\.\d{4}", first_row)  # Find date pattern (DD.MM.YYYY)
#         event_date = datetime.strptime(date_match.group(), "%d.%m.%Y").date() if date_match else None
#         print(f"Extracted Date: {event_date}")

#         # Extract time from the string (HH:MM)
#         time_match = re.search(r"\b\d{1,2}:\d{2}\b", first_row)  # Find time pattern (HH:MM)
#         event_time = time_match.group() if time_match else None
#         print(f"Extracted Time: {event_time}")

#         # Extract category by removing date and time
#         category = first_row
#         if date_match:
#             category = category.replace(date_match.group(), "").strip()  # Remove date
#         if time_match:
#             category = category.replace(time_match.group(), "").strip()  # Remove time
        
#         # Extract track length, hurdles, and gender
#         track_length = re.search(r"\b\d{2,4}m\b", category)
#         track_length = track_length.group(0) if track_length else None
#         hurdles = "Hurdles" if "Hurdles" in category else None
#         gender = "Men" if "Men" in category else "Women" if "Women" in category else None
        
#         # Further clean category
#         if track_length:
#             category = category.replace(track_length, "").strip()
#         if hurdles:
#             category = category.replace("Hurdles", "").strip()
#         if gender:
#             category = category.replace(gender, "").strip()

#         print(f"Event Details - Track Length: {track_length}, Hurdles: {hurdles}, Gender: {gender}, Category: {category}")

#         # Extract WR and MR
#         wr_info = metadata_row.iloc[1].dropna().tolist()
#         mr_info = metadata_row.iloc[2].dropna().tolist()

#         print(f"World Record Row: {wr_info}")
#         print(f"Meeting Record Row: {mr_info}")

#         wr_athlete, wr_nat, wr_city, wr_date = (None, None, None, None)
#         if len(wr_info) >= 5:
#             wr_athlete, wr_nat, wr_city = wr_info[2:5]
#             wr_date = wr_info[5] if isinstance(wr_info[5], str) else (wr_info[5].date() if isinstance(wr_info[5], pd.Timestamp) else None)
        
#         mr_athlete, mr_nat, mr_date = (None, None, None)  # No `mr_city`
#         if len(mr_info) >= 4:
#             mr_athlete, mr_nat = mr_info[2:4]
#             mr_date = mr_info[4] if isinstance(mr_info[4], str) else (mr_info[4].date() if isinstance(mr_info[4], pd.Timestamp) else None)

#         print(f"WR: {wr_athlete}, {wr_nat}, {wr_city}, {wr_date}")
#         print(f"MR: {mr_athlete}, {mr_nat}, {mr_date} (No city)")

#         # Extract results table
#         table = df.iloc[start+1:end].copy()
#         table.columns = df.iloc[start]
#         table.columns = table.columns.astype(str).str.strip()

#         try:
#             table = table[columns_of_interest]
#         except KeyError:
#             print("Skipping table due to missing columns.")
#             continue
        
#         table = table.dropna(how='all')

#         # Extract wind information dynamically for each table
#         wind = None
#         wind_info = str(df.iloc[end, 0])  # Get wind info from the row where 'judge:' is found
#         if "wind:" in wind_info:
#             wind = wind_info.split("wind:")[1].strip()
#         print(f"Wind for this table: {wind}")

#         # Add extracted metadata to the table
#         table['track_length'] = track_length
#         table['hurdles'] = hurdles
#         table['gender'] = gender
#         table['category'] = category
#         table['date'] = event_date
#         table['time'] = event_time  # New column for time
#         table['judge'] = judge  # Assign the globally extracted judge
#         table['wind'] = wind  # Assign the table-specific wind value
#         table['wr_athlete'] = wr_athlete
#         table['wr_nat'] = wr_nat
#         table['wr_city'] = wr_city
#         table['wr_date'] = wr_date
#         table['mr_athlete'] = mr_athlete
#         table['mr_nat'] = mr_nat
#         table['mr_date'] = mr_date  # No `mr_city`

#         # Split athlete names
#         table['athlete_surname'] = table['athlete'].str.split(',').str[0].str.strip().str.lower()
#         table['athlete_name'] = table['athlete'].str.split(',').str[1].str.strip().str.lower()
#         table = table.drop(columns=['athlete'])

#         print(f"Final table shape: {table.shape}")
#         tables.append(table)

#         metadata_list.append({
#             "track_length": track_length,
#             "hurdles": hurdles,
#             "gender": gender,
#             "category": category,
#             "date": event_date,
#             "time": event_time,  # New column for time
#             "judge": judge,  # Assigned globally
#             "wind": wind,  # Wind now specific to each table
#             "wr_athlete": wr_athlete,
#             "wr_nat": wr_nat,
#             "wr_city": wr_city,
#             "wr_date": wr_date,
#             "mr_athlete": mr_athlete,
#             "mr_nat": mr_nat,
#             "mr_date": mr_date,  # No `mr_city`
#         })

#     print("\n✅ Extraction completed.")
#     return tables, metadata_list


In [37]:
def extract_table_info(df):
    columns_of_interest = ['rank', 'BIB', 'athlete', 'nat', 'finish time', 'reaction time']
    tables = []
    metadata_list = []

    # Extract judge's name once from the entire dataset
    judge_info_row = df[df.iloc[:, 0].astype(str).str.contains('judge:', na=False)]
    judge = None
    if not judge_info_row.empty:
        judge_wind_info = str(judge_info_row.iloc[0, 0])  # First occurrence
        if "judge:" in judge_wind_info:
            judge = judge_wind_info.split("judge:")[1].split("wind:")[0].strip().split("(")[0].strip()
    print(f"Extracted Judge: {judge}")  # Debugging Output

    start_indices = df[df.iloc[:, 0].astype(str).str.contains('rank', na=False)].index
    end_indices = df[df.iloc[:, 0].astype(str).str.contains('judge', na=False)].index

    for start, end in zip(start_indices, end_indices):
        print(f"\nProcessing table from row {start} to {end}")

        # Extract metadata
        metadata_row = df.iloc[start-3:start].reset_index(drop=True)
        first_row = metadata_row.iloc[0, 0].replace("Result List", "").strip()
        print(f"First Row Extracted: {first_row}")

        # Extract date
        date_match = re.search(r"\d{2}\.\d{2}\.\d{4}", first_row)
        event_date = datetime.strptime(date_match.group(), "%d.%m.%Y").date() if date_match else None
        print(f"Extracted Date: {event_date}")

        # Extract time
        time_match = re.search(r"\b\d{1,2}:\d{2}\b", first_row)
        event_time = time_match.group() if time_match else None
        print(f"Extracted Time: {event_time}")

        # Extract category
        category = first_row
        if date_match:
            category = category.replace(date_match.group(), "").strip()
        if time_match:
            category = category.replace(time_match.group(), "").strip()

        # Extract event details
        track_length = re.search(r"\b\d{2,4}m\b", category)
        track_length = track_length.group(0) if track_length else None
        hurdles = "Hurdles" if "Hurdles" in category else None
        gender = "Men" if "Men" in category else "Women" if "Women" in category else None

        if track_length:
            category = category.replace(track_length, "").strip()
        if hurdles:
            category = category.replace("Hurdles", "").strip()
        if gender:
            category = category.replace(gender, "").strip()

        print(f"Event Details - Track Length: {track_length}, Hurdles: {hurdles}, Gender: {gender}, Category: {category}")

        # Extract WR and MR info
        wr_info = metadata_row.iloc[1].dropna().tolist()
        mr_info = metadata_row.iloc[2].dropna().tolist()

        print(f"World Record Row: {wr_info}")
        print(f"Meeting Record Row: {mr_info}")

        wr_time, wr_athlete, wr_nat, wr_city, wr_date = (None, None, None, None, None)
        if len(wr_info) >= 6:
            wr_time = wr_info[1]
            wr_athlete, wr_nat, wr_city = wr_info[2:5]
            wr_date = wr_info[5] if isinstance(wr_info[5], str) else (wr_info[5].date() if isinstance(wr_info[5], pd.Timestamp) else None)

        mr_time, mr_athlete, mr_nat, mr_date = (None, None, None, None)
        if len(mr_info) >= 5:
            mr_time = mr_info[1]
            mr_athlete, mr_nat = mr_info[2:4]
            mr_date = mr_info[4] if isinstance(mr_info[4], str) else (mr_info[4].date() if isinstance(mr_info[4], pd.Timestamp) else None)

        print(f"WR: {wr_time}, {wr_athlete}, {wr_nat}, {wr_city}, {wr_date}")
        print(f"MR: {mr_time}, {mr_athlete}, {mr_nat}, {mr_date} (No city)")

        # Extract results table
        table = df.iloc[start+1:end].copy()
        table.columns = df.iloc[start]
        table.columns = table.columns.astype(str).str.strip()

        try:
            table = table[columns_of_interest]
        except KeyError:
            print("Skipping table due to missing columns.")
            continue
        
        table = table.dropna(how='all')

        # Extract wind information for this table
        wind = None
        wind_info = str(df.iloc[end, 0])
        if "wind:" in wind_info:
            wind = wind_info.split("wind:")[1].strip()
        print(f"Wind for this table: {wind}")

        # Add metadata columns to the result table
        table['track_length'] = track_length
        table['hurdles'] = hurdles
        table['gender'] = gender
        table['category'] = category
        table['date'] = event_date
        table['time'] = event_time
        table['judge'] = judge
        table['wind'] = wind

        table['wr_time'] = wr_time
        table['wr_athlete'] = wr_athlete
        table['wr_nat'] = wr_nat
        table['wr_city'] = wr_city
        table['wr_date'] = wr_date

        table['mr_time'] = mr_time
        table['mr_athlete'] = mr_athlete
        table['mr_nat'] = mr_nat
        table['mr_date'] = mr_date

        # Split athlete name
        table['athlete_surname'] = table['athlete'].str.split(',').str[0].str.strip().str.lower()
        table['athlete_name'] = table['athlete'].str.split(',').str[1].str.strip().str.lower()
        table = table.drop(columns=['athlete'])

        print(f"Final table shape: {table.shape}")
        tables.append(table)

        metadata_list.append({
            "track_length": track_length,
            "hurdles": hurdles,
            "gender": gender,
            "category": category,
            "date": event_date,
            "time": event_time,
            "judge": judge,
            "wind": wind,
            "wr_time": wr_time,
            "wr_athlete": wr_athlete,
            "wr_nat": wr_nat,
            "wr_city": wr_city,
            "wr_date": wr_date,
            "mr_time": mr_time,
            "mr_athlete": mr_athlete,
            "mr_nat": mr_nat,
            "mr_date": mr_date,
        })

    print("\n✅ Extraction completed.")
    return tables, metadata_list


In [38]:
# Extract the tables and metadata
tables, metadata_list = extract_table_info(df)

# Merge all tables into one final DataFrame
if tables:  # Ensure there is data before concatenating
    df_final = pd.concat(tables, ignore_index=True)
    
    # Print the final DataFrame
    print("Final DataFrame (df_final):")
    display(df_final.head(20))  # Show only the first 20 rows
else:
    print("No valid tables were extracted.")


Extracted Judge: Frederic Stocco

Processing table from row 3 to 12
First Row Extracted: 400m Hurdles Men   B-Series 17.08.2001  18:30
Extracted Date: 2001-08-17
Extracted Time: 18:30
Event Details - Track Length: 400m, Hurdles: Hurdles, Gender: Men, Category: B-Series
World Record Row: ['World Record', 46.78, 'Kevin Young', 'USA', 'Barcelone', Timestamp('1992-08-06 00:00:00')]
Meeting Record Row: ['Meeting Record', 47.1, 'Samuel Matete', 'ZAM', Timestamp('1991-08-07 00:00:00')]
WR: 46.78, Kevin Young, USA, Barcelone, 1992-08-06
MR: 47.1, Samuel Matete, ZAM, 1991-08-07 (No city)
Wind for this table: None
Final table shape: (8, 24)

Processing table from row 19 to 28
First Row Extracted: 100m Women   B-Series  17.08.2001  18:40
Extracted Date: 2001-08-17
Extracted Time: 18:40
Event Details - Track Length: 100m, Hurdles: None, Gender: Women, Category: B-Series
World Record Row: ['World Record', 10.49, 'Florence Griffith-Joyner', 'USA', 'Indianapolis', Timestamp('1988-07-16 00:00:00')]
Me

,rank,BIB,nat,finish time,reaction time,track_length,hurdles,gender,category,date,...,wr_athlete,wr_nat,wr_city,wr_date,mr_time,mr_athlete,mr_nat,mr_date,athlete_surname,athlete_name
0,1,213,RUS,48.58,0.262,400m,Hurdles,Men,B-Series,2001-08-17,...,Kevin Young,USA,Barcelone,1992-08-06,47.10,Samuel Matete,ZAM,1991-08-07,gorban,boris
1,2,218,POL,48.69,0.199,400m,Hurdles,Men,B-Series,2001-08-17,...,Kevin Young,USA,Barcelone,1992-08-06,47.10,Samuel Matete,ZAM,1991-08-07,plawgo,marek
2,3,467,JAM,49.08,0.29,400m,Hurdles,Men,B-Series,2001-08-17,...,Kevin Young,USA,Barcelone,1992-08-06,47.10,Samuel Matete,ZAM,1991-08-07,weakly,ian
3,4,217,CZE,49.09,0.312,400m,Hurdles,Men,B-Series,2001-08-17,...,Kevin Young,USA,Barcelone,1992-08-06,47.10,Samuel Matete,ZAM,1991-08-07,muzik,jiri
4,5,215,RUS,49.13,0.262,400m,Hurdles,Men,B-Series,2001-08-17,...,Kevin Young,USA,Barcelone,1992-08-06,47.10,Samuel Matete,ZAM,1991-08-07,machtchenko,rouslan
5,6,228,RSA,49.25,0.186,400m,Hurdles,Men,B-Series,2001-08-17,...,Kevin Young,USA,Barcelone,1992-08-06,47.10,Samuel Matete,ZAM,1991-08-07,herbert,llewellyn
6,7,226,ITA,49.41,0.177,400m,Hurdles,Men,B-Series,2001-08-17,...,Kevin Young,USA,Barcelone,1992-08-06,47.10,Samuel Matete,ZAM,1991-08-07,ottoz,laurent
7,8,229,FRA,49.42,0.175,400m,Hurdles,Men,B-Series,2001-08-17,...,Kevin Young,USA,Barcelone,1992-08-06,47.10,Samuel Matete,ZAM,1991-08-07,theodore,olivier jean
8,1,260,NGR,11.06,0.17,100m,None,Women,B-Series,2001-08-17,...,Florence Griffith-Joyner,USA,Indianapolis,1988-07-16,10.76,Evelyn Ashford,USA,1984-08-22,ojokolo,endurance
9,2,261,NGR,11.26,0.2,100m,None,Women,B-Series,2001-08-17,...,Florence Griffith-Joyner,USA,Indianapolis,1988-07-16,10.76,Evelyn Ashford,USA,1984-08-22,onyali,mary


In [39]:
df_final['date'] = pd.to_datetime(df_final['date'], errors='coerce').dt.strftime("%d/%m/%Y")

In [40]:
df_final['time'] = pd.to_datetime(df_final['time'], errors='coerce').dt.strftime("%H:%M")

C:\Users\ALESSANDRO\AppData\Local\Temp\ipykernel_10256\4072969283.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_final['time'] = pd.to_datetime(df_final['time'], errors='coerce').dt.strftime("%H:%M")


In [41]:
condition = (df_final['date'] == "17/08/2001") & (df_final['time'] == "21:55")

df_final.loc[condition, 'wr_athlete'] = "Kim Batten"
df_final.loc[condition, 'wr_nat'] = "USA"
df_final.loc[condition, 'wr_city'] = "Goteborg"
df_final.loc[condition, 'wr_time'] = "52.61"
df_final.loc[condition, 'wr_date'] = "11.08.1995"


In [42]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\false_start\data\out\kate\2001_zurich_extracted.xlsx'
df_final.to_excel(file_path, index=False)
